**Relevant resources:**

* [Unsloth Github](https://github.com/unslothai/unsloth)
* [Llama 3.1 Base 8b Model](https://huggingface.co/unsloth/Meta-Llama-3.1-8B-bnb-4bit)
* [Llama 3.1 Instruct 8b Model](https://huggingface.co/unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit)
* [Llama-3.1 8b + Unsloth 2x faster finetuning](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing)
* [Llama-3 8b Instruct Unsloth 2x faster finetuning](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing)

**Load model from Unsloth**

In [ ]:
from unsloth import FastLanguageModel
import torch
import gc
import os 

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    # attn_implementation="flash_attention_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

**Add LoRA adapters**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Load Data

* [Loading dataset from different formats](https://huggingface.co/docs/datasets/en/loading)
* [Loading dataset from SQLite3 database](https://huggingface.co/docs/datasets/main/en/tabular_load#sqlite)
* [Llama 3.1 Instruct Prompt Format](https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#llama-3.1-instruct)

In [ ]:
from datasets import Dataset, load_dataset

instructions = """
You are a helpful assistant.
"""

def format_prompts(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_txt, output_txt in zip(inputs, outputs):
        text = [
            {"role": "system", "content": instructions},
            {"role": "user", "content": input_txt},
            {"role": "assistant", "content": output_txt}
        ]
        texts.append(tokenizer.apply_chat_template(text, tokenize=False))
    return {"text": texts,}

ds_file_path = "/mnt/d/Code/reddit/dataset/combined.json"
dataset = load_dataset("json", data_files=ds_file_path, split="train")
dataset = dataset.shuffle(seed=42)
dataset = dataset.map(format_prompts, batched = True,)

In [ ]:
# print(dataset[0]["text"])

### Train model

* [Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # train_dataset = train_dataset,
    # eval_dataset = eval_dataset, # Set this if using evaluation dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 6,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 150,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        save_steps = 250,
        save_total_limit = 8,
        # eval_strategy = "steps", # Set this if using evaluation dataset
        # eval_steps = 100, # Set this if using evaluation dataset
        output_dir = "outputs",
    ),
)

**Show current memory stats**

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

**Train model**

In [ ]:
trainer_stats = trainer.train()
# trainer_stats = trainer.train(resume_from_checkpoint=True)

**Show final memory and time stats**

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

**Show training loss over time**

In [ ]:
import matplotlib.pyplot as plt

# Extract the loss values
loss_values = trainer.state.log_history

# Extracting only the loss from each log entry
training_loss = [entry['loss'] for entry in loss_values if 'loss' in entry]

# Plotting the loss over time
plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.xlabel('Training Steps')
plt.ylabel('Loss')
plt.title('Train/Loss')
plt.legend()
plt.grid(True)
plt.show()

**Save finetuned model**

In [ ]:
# model.save_pretrained("model")
trainer.save_model("model")
trainer.save_state()
# print("model saved")

### Try inference

In [ ]:
# instructions = "You are a helpful AI assistant."

prompt = ""

msgs = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": prompt},
]
FastLanguageModel.for_inference(model)
inputs = tokenizer.apply_chat_template(
    msgs,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt"
).to("cuda")
attention_mask = (inputs != tokenizer.pad_token_id).int()

outputs = model.generate(
    input_ids = inputs, 
    attention_mask = attention_mask, 
    min_new_tokens = 64, 
    max_new_tokens = 1024,
    temperature = 0.8,
    repetition_penalty=1.25,
    top_p = 0.9,
    use_cache = True
)
print(tokenizer.batch_decode(outputs)[0])